In [2]:
import sys
# sys.path.insert(0, '..')

from mxnet import nd
import random

with open('35.txt', 'r') as f:
    lines = f.readlines()
    raw_dataset = ' '.join(' '.join(lines).lower().split())

print('number of characters: ', len(raw_dataset))
print(raw_dataset[0:70])

number of characters:  197070
the time machine, by h. g. wells [1898] i the time traveller (for so i


In [4]:
idx_to_char = list(set(raw_dataset))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
print(char_to_idx)

{'k': 0, '9': 1, ',': 2, '?': 3, 'c': 4, '4': 5, "'": 6, '.': 7, 'j': 8, 'b': 9, 'r': 10, '%': 11, 'p': 12, 'm': 13, 'n': 14, '2': 15, '0': 16, 'x': 17, '_': 18, 'q': 19, 'i': 20, 'g': 21, 'v': 22, 'a': 23, 't': 24, ')': 25, '[': 26, 'f': 27, 'o': 28, '"': 29, '*': 30, 'd': 31, '7': 32, 'z': 33, '@': 34, '5': 35, 'u': 36, 's': 37, '!': 38, 'w': 39, 'e': 40, '$': 41, ';': 42, '3': 43, ':': 44, '1': 45, '8': 46, '/': 47, '6': 48, ' ': 49, 'y': 50, 'h': 51, 'l': 52, '(': 53, '-': 54, ']': 55}


In [5]:
corpus_indices = [char_to_idx[char] for char in raw_dataset]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

chars: the time machine, by
indices: [24, 51, 40, 49, 24, 20, 13, 40, 49, 13, 23, 4, 51, 20, 14, 40, 2, 49, 9, 50]


In [6]:
# This function is saved in the d2l package for future use
def data_iter_random(corpus_indices, batch_size, num_steps, ctx=None):
    # Offset for the iterator over the data for uniform starts
    offset = int(random.uniform(0,num_steps))
    corpus_indices = corpus_indices[offset:]
    # Subtract 1 extra since we need to account for the sequence length
    num_examples = ((len(corpus_indices) - 1) // num_steps) - 1
    # Discard half empty batches
    num_batches = num_examples // batch_size
    example_indices = list(range(0, num_examples * num_steps, num_steps))
    random.shuffle(example_indices)

    # This returns a sequence of the length num_steps starting from pos
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]

    for i in range(0, batch_size * num_batches, batch_size):
        # Batch_size indicates the random examples read each time
        batch_indices = example_indices[i:(i+batch_size)]
        X = [_data(j) for j in batch_indices]
        Y = [_data(j + 1) for j in batch_indices]
        yield nd.array(X, ctx), nd.array(Y, ctx)

In [29]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=3):
    print('X: ', X, '\nY:', Y)


X:  
[[13. 14. 15.]
 [10. 11. 12.]]
<NDArray 2x3 @cpu(0)> 
Y: 
[[14. 15. 16.]
 [11. 12. 13.]]
<NDArray 2x3 @cpu(0)>
X:  
[[4. 5. 6.]
 [1. 2. 3.]]
<NDArray 2x3 @cpu(0)> 
Y: 
[[5. 6. 7.]
 [2. 3. 4.]]
<NDArray 2x3 @cpu(0)>
X:  
[[22. 23. 24.]
 [ 7.  8.  9.]]
<NDArray 2x3 @cpu(0)> 
Y: 
[[23. 24. 25.]
 [ 8.  9. 10.]]
<NDArray 2x3 @cpu(0)>
X:  
[[16. 17. 18.]
 [19. 20. 21.]]
<NDArray 2x3 @cpu(0)> 
Y: 
[[17. 18. 19.]
 [20. 21. 22.]]
<NDArray 2x3 @cpu(0)>


In [30]:
# This function is saved in the d2l package for future use
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    # Offset for the iterator over the data for uniform starts
    offset = int(random.uniform(0,num_steps))
    # Slice out data - ignore num_steps and just wrap around
    num_indices = ((len(corpus_indices) - offset) // batch_size) * batch_size
    indices = nd.array(corpus_indices[offset:(offset + num_indices)], ctx=ctx)
    indices = indices.reshape((batch_size,-1))
    # Need to leave one last token since targets are shifted by 1
    num_epochs = ((num_indices // batch_size) - 1) // num_steps

    for i in range(0, num_epochs * num_steps, num_steps):
        X = indices[:,i:(i+num_steps)]
        Y = indices[:,(i+1):(i+1+num_steps)]
        yield X, Y

In [32]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=5):
    print('X: ', X, '\nY:', Y)

X:  
[[ 1.  2.  3.  4.  5.]
 [15. 16. 17. 18. 19.]]
<NDArray 2x5 @cpu(0)> 
Y: 
[[ 2.  3.  4.  5.  6.]
 [16. 17. 18. 19. 20.]]
<NDArray 2x5 @cpu(0)>
X:  
[[ 6.  7.  8.  9. 10.]
 [20. 21. 22. 23. 24.]]
<NDArray 2x5 @cpu(0)> 
Y: 
[[ 7.  8.  9. 10. 11.]
 [21. 22. 23. 24. 25.]]
<NDArray 2x5 @cpu(0)>
